In [26]:
import csv
reader = csv.reader(open(r"D:\InterestingProgramming\python\Pytorch深度学习实战\data\p1ch4\bike-sharing-dataset\hour-fixed.csv"), delimiter=',')
bikeList = next(reader)
bikeList = next(reader)
bikeList[1]

import torch
import numpy as np

bikes_numpy = np.loadtxt(
    r"D:\InterestingProgramming\python\Pytorch深度学习实战\data\p1ch4\bike-sharing-dataset\hour-fixed.csv",
    dtype=np.float32,
    delimiter=",",
    skiprows=1,
    converters={1: lambda x: float(x[8:10])}
)
bikes = torch.from_numpy(bikes_numpy)
bikes

bikes.shape, bikes.stride()


(torch.Size([17520, 17]), (17, 1))

In [27]:
# 使用view来重置张量的布局
daily_bikes = bikes.view(730, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [28]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [29]:
first_day = bikes[:24].long()
weather_hot = torch.zeros(first_day.shape[0], 4)
weather_hot.scatter_(
    dim=1,
    index=first_day[:,9].unsqueeze(1).long()-1, # 1还是-1都可以
    value=1.0
)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [30]:
torch.cat((bikes[:24], weather_hot), dim=1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [31]:
# 为重塑后的daily_weather附加one_hot
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[-1])
daily_weather_onehot.shape, daily_bikes.shape

(torch.Size([730, 4, 24]), torch.Size([730, 17, 24]))

In [32]:
daily_weather_onehot.scatter_(
    dim=1,
    index=daily_bikes[:,9,:].long().unsqueeze(1)-1, value=1.0)
daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [33]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

In [34]:
daily_bikes[:,9,:] = (daily_bikes[:, 9, :]-1.0)/3.0
daily_bikes[:,9,:].max()

tensor(1.)

In [35]:
# 温度映射到0-1
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = (daily_bikes[:, 10, :] - temp_min) / (temp_max-temp_min)

In [36]:
daily_bikes[:, 10, :].max()

tensor(1.)